## Activ4a dataset decoding

The datasets of Activ4a (phs002694) were provided as CSV files that contained encoded data. For the BioData Catalyst PIC-SURE ETL pipeline, these files need to be in the decoded format. The purpose of this notebook is to use the Data Dictionary, provided in Excel format, to decode the CSV files and save them to the S3 bucket.

### Do imports, set directory paths, and define functions

In [ ]:
# Do imports
import pandas as pd
import glob

In [ ]:
# Set directories and get file information
data_dict_path = "/home/ec2-user/SageMaker/pic-sure-metadata-curation/activ4a/input/Data Dictionary 2022-01-04.xlsx"
file_dir = "/home/ec2-user/studies/ALL-avillach-73-bdcatalyst-etl/activ4a/development_raw_data/*"
out_dir = "/home/ec2-user/studies/ALL-avillach-73-bdcatalyst-etl/activ4a/decoded_data/"
file_prefixes = [i.split('/')[-1].strip(".csv") for i in glob.glob(file_dir)]

In [ ]:
def decode_data(data_dict_path, file_prefixes, file_dir, out_dir, to_pop=None):
    for f in file_prefixes:
        data_dict = pd.read_excel(data_dict_path, 
                      sheet_name = f, 
                      engine = "openpyxl")
        data_dict = data_dict.dropna(subset=["code"]).reset_index(drop=True)
        data_dict['code'] = data_dict['code'].astype('str') 
        full_dict = make_decode_dict(data_dict, to_pop)
        try:
            data = pd.read_csv(file_dir.strip("*")+f+".csv")
        except:
            print("Review data of", f)
            continue
        try: 
            decoded_data = data.replace(full_dict)
        except:
            print("Review data dictionary of", f)
            print("Unsuccessful")
            continue
        out_file = out_dir+f+".csv"
        decoded_data.to_csv(out_file, index=False)
        print("Decoded dataset", out_file)
        
def make_decode_dict(df, pop_val):
    full_dict = {}
    for var in df.VARNAME:
        data = df.code[df.VARNAME == var].values[0]
        list_data = data.split("|")
        mini_dict = {}
        for info in list_data:
            mapping = info.split("=")
            try:
                encoded_val = int(mapping[0])
            except:
                encoded_val = mapping[0].strip().strip("'")
            if (pop_val is not None and encoded_val not in pop_val) or pop_val is None:
                decoded_val = mapping[1].strip().strip("'")
                mini_dict.update({encoded_val: decoded_val})
        full_dict.update({var: mini_dict})
    return full_dict

### Use functions to decode the data
Note that based on the output, there were some files that we were not able to decode without some manual curation. These were:
- ADJ_STROKE
- BIOREP
- CMBL
- CONS
- HS
- IE
- LBRES
- OUTCOME
- REHOSP
- SAMPLE
- SD
- STATUS

In [ ]:
decode_data(data_dict_path, file_prefixes, file_dir, out_dir)

### Manual review of problematic data dictionaries
After reviewing the data dictionaries and data files by hand, there were some decoded values that were not in the data and thus causing problems. These included `.U`, `.A`, and `.D`. Since these are strings while the type of the data were numeric, this was causing problems in the `replace` function. Therefore, the functions were adjusted to include an option to exclude these parts of the data dictionary.

In [ ]:
# Need to review data dictionary ADJ_STROKE (pop .U), BIOREP (pop .A), CONS (pop .U and .D), HS (pop .U), 
# IE (pop .A), LBRES (pop .D), SAMPLE (pop .A), SD (pop .D), STATUS (pop .U)

In [ ]:
u_pop_prefixes = ['ADJ_STROKE', 'HS', 'STATUS', "MH", "OUTCOME"]
a_pop_prefixes = ['BIOREP', 'IE', 'SAMPLE']
d_pop_prefixes = ['LBRES',  'SD']
ud_pop_prefixes = ['CONS']
aud_pop_prefixes = ["AE"]
ad_pop_prefixes = ["VS"]
decode_data(data_dict_path, u_pop_prefixes, file_dir, out_dir, to_pop=['.U'])
decode_data(data_dict_path, a_pop_prefixes, file_dir, out_dir, to_pop=['.A'])
decode_data(data_dict_path, d_pop_prefixes, file_dir, out_dir, to_pop=['.D'])
decode_data(data_dict_path, ud_pop_prefixes, file_dir, out_dir, to_pop=['.U', '.D'])
decode_data(data_dict_path, aud_pop_prefixes, file_dir, out_dir, to_pop=['.A', '.U', '.D'])
decode_data(data_dict_path, ad_pop_prefixes, file_dir, out_dir, to_pop=['.A', '.D'])

### Manual review of problematic data
After attempting to review the data files on the S3 bucket through Service Workbench, it became obvious that the issue with some of these data files were the encoding. This was resolved by manually saving these files in the "CSV UTF-8" format and reuploading them to the S3 bucket.

In [ ]:
decode_data(data_dict_path, ["CMBL"], file_dir, out_dir)

In [ ]:
decode_data(data_dict_path, ["OUTCOME"], file_dir, out_dir)

### Confirm that data was decoded as expected
Since there were some unique problems with some of the data files and dictionaries, compare the raw and decoded CSV files to see if they changed.
The following files need further investigation:
- AE
- MH
- MPRCT
- OUTCOME
- VS
- WITHDSC

In [ ]:
def check_decode(file_dir, out_dir, file_prefixes, out=False):
    for f in file_prefixes:
        raw_df = pd.read_csv(file_dir.strip("*")+f+".csv")
        decode_df = pd.read_csv(out_dir+f+".csv")
        comp = decode_df.compare(raw_df)
        if comp.shape[0] == 0:
            print(f, "not decoded properly")
        else:
            print("Successful", f)
        if out:
            return raw_df

In [ ]:
check_decode(file_dir, out_dir, file_prefixes)

In [ ]:
# Manually check each file
#ae = check_decode(file_dir, out_dir, ["AE"], True)
#cmbl = check_decode(file_dir, out_dir, ['CMBL'], True)
#mh = check_decode(file_dir, out_dir, ['MH'], True)
#mprct = check_decode(file_dir, out_dir, ['MPRCT'], True)
#outcome = check_decode(file_dir, out_dir, ['OUTCOME'], True)
#rehosp = check_decode(file_dir, out_dir, ['REHOSP'], True)
#vs = check_decode(file_dir, out_dir, ['VS'], True)
#withdsc = check_decode(file_dir, out_dir, ['WITHDSC'], True)
#withdsc

### Results from check:
- Need to fix cmbl, outcome, rehosp
- AE, MH, VS data dictionary and data file do not match
    - Note: there were two files hosted on Gen3 for these specific files. This was fixed by uploading the other version of the file to the S3 bucket and rerunning the code.
- MPRCT there is no data to decode, all variables are strings or numeric. This file is OK.
- WITHDSC is an empty dataframe, therefore no data to decode

In [ ]:
# Manual check of complicated files
#f = 'OUTCOME'
#to_pop = None
#data_dict = pd.read_excel(data_dict_path, sheet_name = f, engine = "openpyxl")
#data_dict = data_dict.dropna(subset=["code"]).reset_index(drop=True)
#data_dict['code'] = data_dict['code'].astype('str') 
#full_dict = make_decode_dict(data_dict, to_pop)
#full_dict = {}
#for var in data_dict.VARNAME:
#    data = data_dict.code[data_dict.VARNAME == var].values[0]
#    list_data = data.split("|")
#    print(list_data)
#    mini_dict = {}
#    #for info in list_data:
#    #    mapping = info.split("=")
#    #    try:
#    #        encoded_val = int(mapping[0])
#    #    except:
#    #        encoded_val = mapping[0].strip().strip("'")
#    #    if (to_pop is not None and encoded_val not in to_pop) or to_pop is None:
#    #        decoded_val = mapping[1].strip().strip("'")
#    #        mini_dict.update({encoded_val: decoded_val})
#    ##print(encoded_val, decoded_val)
#    #full_dict.update({var: mini_dict})
#print(full_dict)
#try:
#    data = pd.read_csv(file_dir.strip("*")+f+".csv")
#except:
#    print("Review data of", f)
#try: 
#    decoded_data = data.replace(full_dict)
#except:
#    print("Review data dictionary of", f)
#    print("Unsuccessful")
#out_file = out_dir+f+".csv"
#decoded_data.to_csv(out_file, index=False)
#print("Decoded dataset", out_file)